In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.preprocessing import StandardScaler
from finpak.data.fetchers.yahoo import download_multiple_tickers


def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# Use this device throughout your code
device = get_device()
print(f"Using device: {device}")


Using device: mps


In [25]:


tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = '2000-04-01'
end_date = '2024-08-31'

# Download historical data for the tickers
data_df = download_multiple_tickers(tickers, start_date, end_date)
data_df = data_df.loc[:,'Adj Close'] # Extract from multi-index dataframe
data_df.tail(8)


data_df.head(2)


data_df['AAPL'].values[-10:]

[*********************100%***********************]  3 of 3 completed


array([225.88999939, 226.50999451, 226.3999939 , 224.52999878,
       226.83999634, 227.17999268, 228.02999878, 226.49000549,
       229.78999329, 229.        ])

In [26]:
prices = data_df['AAPL'].values 
prices[-10:]


# # Calculate percentage changes
# pct_changes = np.diff(prices) / prices[:-1]

# # Optionally adjust by volatility
# adjust_by_volatility = False # True
# if adjust_by_volatility:
#     volatility = np.std(pct_changes)
#     pct_changes = pct_changes / volatility


# pct_changes[:10]


array([225.88999939, 226.50999451, 226.3999939 , 224.52999878,
       226.83999634, 227.17999268, 228.02999878, 226.49000549,
       229.78999329, 229.        ])

In [27]:
"""
- **Create Sequences:**
  - Decide on a sequence length (e.g., 50).
  - Convert the percentage changes into input-output pairs where each input is a sequence of `sequence_length` steps, and the output is the next value.
"""

# After downloading the data

def calculate_features(prices):
    # 1-day percentage change
    pct_change_1d = np.diff(prices) / prices[:-1]
    pct_change_1d = np.insert(pct_change_1d, 0, 0)  # Insert 0 at the beginning for alignment
    
    # Percentage difference from 30-day moving average
    ma_30 = pd.Series(prices).rolling(window=30).mean()
    pct_diff_ma30 = (prices - ma_30) / ma_30
    
    # 5-day percentage change
    pct_change_5d = (prices[5:] - prices[:-5]) / prices[:-5]
    pct_change_5d = np.pad(pct_change_5d, (5, 0), mode='constant', constant_values=0)
    
    # 15-day percentage change
    pct_change_15d = (prices[15:] - prices[:-15]) / prices[:-15]
    pct_change_15d = np.pad(pct_change_15d, (15, 0), mode='constant', constant_values=0)
    
    return np.column_stack((pct_change_1d, pct_diff_ma30, pct_change_5d, pct_change_15d))

# Calculate features
features = calculate_features(prices)

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences
sequence_length = 10
X = []
y = []

for i in range(len(scaled_features) - sequence_length):
    X.append(scaled_features[i:i + sequence_length])
    y.append(scaled_features[i + sequence_length, 0])  # Still predicting 1-day percentage change

X = np.array(X)
y = np.array(y)

In [28]:
# adjust for 30-day moving average
X = X[30:]
y = y[30:]


In [29]:
X[0], y[0]


(array([[ 1.86999057, -1.49905685, -0.06743923, -2.11510724],
        [-1.73652598, -1.89912034,  0.28575537, -1.98479921],
        [-0.30412708, -1.87230219, -0.4981022 , -2.42838803],
        [-2.81971483, -2.54555765, -2.54348646, -2.83292709],
        [-1.83640201, -2.8858714 , -2.21582963, -3.2049885 ],
        [-1.94581077, -3.24999036, -3.72339521, -3.15585151],
        [ 0.85434742, -2.92972937, -2.70518106, -2.78503818],
        [-0.24813835, -2.91067053, -2.68255723, -2.49883621],
        [-0.47122499, -2.93056074, -1.67033229, -2.76926462],
        [ 0.51936111, -2.70760036, -0.61993824, -2.42570357]]),
 np.float64(-1.7316410359523122))

In [42]:
class StockDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.tensor(sequences, dtype=torch.float32).to(device)
        self.targets = torch.tensor(targets, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]


class SwishActivation(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class LSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMLayer, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.swish = SwishActivation()

    def forward(self, x, hidden=None):
        out, hidden = self.lstm(x, hidden)
        out = self.layer_norm(out)
        out = self.swish(out)
        return out, hidden

class EnhancedLSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_size=2048, num_layers=3):
        super(EnhancedLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.layers = nn.ModuleList([
            LSTMLayer(input_size if i == 0 else hidden_size, hidden_size)
            for i in range(num_layers)
        ])
        
        self.inner_fc = nn.Linear(hidden_size, hidden_size)
        self.inner_ln = nn.LayerNorm(hidden_size)
        self.fc = nn.Linear(hidden_size, 1)
        self.layer_norm_final = nn.LayerNorm(hidden_size)
        self.swish = SwishActivation()

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        hidden = (h0, c0)

        residual = None
        for i, layer in enumerate(self.layers):
            if i == 0:
                out, hidden = layer(x, hidden)
            else:
                layer_input = out if residual is None else out + residual
                out, hidden = layer(layer_input, hidden)
            
            if i > 0:  # Apply residual connection for all layers except the first
                residual = out

        out = self.layer_norm_final(out)
        out = self.swish(out)
        out = self.inner_fc(out)
        out = self.inner_ln(out)
        out = self.swish(out)
        out = self.fc(out[:, -1, :])
        return out.squeeze()

In [31]:
# Create the full dataset
full_dataset = StockDataset(X, y)

# Define split ratios
train_ratio = 0.88
val_ratio = 0.12

# Calculate split indices
train_size = int(len(y) * train_ratio)
val_size = int(len(y) * val_ratio)

# Split the dataset sequentially
train_dataset = torch.utils.data.Subset(full_dataset, range(train_size))
val_dataset = torch.utils.data.Subset(full_dataset, range(train_size, train_size + val_size))

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [43]:
model = EnhancedLSTMModel(input_size=4, hidden_size=888, num_layers=4).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00053)
best_val_loss = float('inf')

In [44]:
num_epochs = 225
patience = 25  # For early stopping
epochs_without_improvement = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = 0
    for sequences, targets in train_loader:
        sequences, targets = sequences.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for sequences, targets in val_loader:
            sequences, targets = sequences.to(device), targets.to(device)
            outputs = model(sequences)
            val_loss = criterion(outputs, targets)
            epoch_val_loss += val_loss.item()
    
    avg_val_loss = epoch_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    # Print statistics
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss: {avg_train_loss:.4f}')
    print(f'Validation Loss: {avg_val_loss:.4f}')
    
    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_without_improvement = 0
        # When saving the model, move it to CPU first
        torch.save(model.to('cpu').state_dict(), 'best_lstm_model.pth')
        model.to(device)  # Move it back to the device if you continue training
        print("New best model saved!")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break
    
    print('-' * 50)

Epoch [1/225]
Train Loss: 1.1783
Validation Loss: 0.5192
New best model saved!
--------------------------------------------------
Epoch [2/225]
Train Loss: 1.0585
Validation Loss: 0.5141
New best model saved!
--------------------------------------------------
Epoch [3/225]
Train Loss: 1.0543
Validation Loss: 0.5161
--------------------------------------------------
Epoch [4/225]
Train Loss: 1.0542
Validation Loss: 0.5181
--------------------------------------------------
Epoch [5/225]
Train Loss: 1.0545
Validation Loss: 0.5161
--------------------------------------------------
Epoch [6/225]
Train Loss: 1.0541
Validation Loss: 0.5156
--------------------------------------------------
Epoch [7/225]
Train Loss: 1.0540
Validation Loss: 0.5152
--------------------------------------------------
Epoch [8/225]
Train Loss: 1.0537
Validation Loss: 0.5153
--------------------------------------------------
Epoch [9/225]
Train Loss: 1.0535
Validation Loss: 0.5150
-----------------------------------

KeyboardInterrupt: 

In [39]:
"""
**Objective:** Use the trained model to make predictions, compare them with actual stock prices, and visualize the results.

- **Load the Trained Model:**
"""

model = EnhancedLSTMModel(input_size=4, hidden_size=2048, num_layers=3).to(device)
model.load_state_dict(torch.load('best_lstm_model.pth', weights_only=True))
model.eval()


"""
- **Autoregressive Prediction:**
"""

# After loading the model and setting it to eval mode

# Define the validation set range
val_start = train_size + 30 # 30-day moving average
val_end = train_size + val_size + 30 # 30-day moving average

# Define prediction parameters
prediction_length = 20  # Number of future steps to predict
prediction_interval = 50  # Space between prediction start points

# Function to make predictions
def make_prediction(start_idx):
    input_seq = scaled_features[start_idx:start_idx + sequence_length]
    input_seq = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).to(device)
    
    predictions = []
    with torch.no_grad():
        for _ in range(prediction_length):
            pred = model(input_seq)
            predictions.append(pred.item())
            
            # Update input sequence for next prediction
            last_features = input_seq[0, -1, :].cpu().numpy()
            new_features = calculate_features(np.array([prices[start_idx + sequence_length - 1] * (1 + pred.item())]))[-1]
            new_features = scaler.transform(new_features.reshape(1, -1))
            new_input = torch.tensor(new_features, dtype=torch.float32).unsqueeze(0).to(device)
            input_seq = torch.cat((input_seq[:, 1:], new_input), dim=1)
    
    return predictions

RuntimeError: Error(s) in loading state_dict for EnhancedLSTMModel:
	Missing key(s) in state_dict: "layers.2.lstm.weight_ih_l0", "layers.2.lstm.weight_hh_l0", "layers.2.lstm.bias_ih_l0", "layers.2.lstm.bias_hh_l0", "layers.2.layer_norm.weight", "layers.2.layer_norm.bias". 
	size mismatch for layers.0.lstm.weight_ih_l0: copying a param with shape torch.Size([32768, 4]) from checkpoint, the shape in current model is torch.Size([8192, 4]).
	size mismatch for layers.0.lstm.weight_hh_l0: copying a param with shape torch.Size([32768, 8192]) from checkpoint, the shape in current model is torch.Size([8192, 2048]).
	size mismatch for layers.0.lstm.bias_ih_l0: copying a param with shape torch.Size([32768]) from checkpoint, the shape in current model is torch.Size([8192]).
	size mismatch for layers.0.lstm.bias_hh_l0: copying a param with shape torch.Size([32768]) from checkpoint, the shape in current model is torch.Size([8192]).
	size mismatch for layers.0.layer_norm.weight: copying a param with 

In [18]:

# Make predictions at intervals
prediction_starts = range(val_start, val_end - sequence_length - prediction_length, prediction_interval)
all_predictions = [make_prediction(start) for start in prediction_starts]

# Denormalize predictions and prepare for plotting
def prepare_prediction_data(predictions, start_idx):
    denorm_predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))[:, 0]
    last_known_price = prices[start_idx + sequence_length - 1]
    predicted_prices = [last_known_price]
    for pred_pct in denorm_predictions:
        next_predicted_price = predicted_prices[-1] * (1 + pred_pct)
        predicted_prices.append(next_predicted_price)
    prediction_dates = pd.date_range(start=data_df.index[start_idx + sequence_length - 1], 
                                     periods=len(predicted_prices), freq='B')
    return prediction_dates, predicted_prices

# Prepare all prediction data
all_prediction_data = [prepare_prediction_data(pred, start) 
                       for pred, start in zip(all_predictions, prediction_starts)]

# Prepare actual price data for the entire validation period
val_dates = data_df.index[val_start:val_end]
val_prices = prices[val_start:val_end]

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 2 has size 5

In [ ]:
# Plotting
plt.figure(figsize=(15, 8))

# Plot actual stock price
plt.plot(val_dates, val_prices, label='Actual Stock Price', color='blue', linewidth=2)

# Plot prediction lines
for dates, predicted_prices in all_prediction_data:
    plt.plot(dates, predicted_prices, color='red', alpha=0.5, linewidth=1)

plt.title('LSTM Model: Multiple Predictions vs Actual Stock Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# Optionally, you can add a legend for predictions
plt.plot([], [], color='red', alpha=0.5, label='Predictions')
plt.legend()

# Save the plot
plt.savefig('lstm_predictions.png')
plt.close()

# Print some statistics
mse = np.mean([((pred[-1] - actual) / actual) ** 2 
               for (_, pred), actual in zip(all_prediction_data, prices[val_start + prediction_length::prediction_interval])])
print(f"Mean Squared Error of predictions: {mse:.4f}")

